In [1]:
import os
from dotenv import load_dotenv
from litellm import completion
from typing import List, Dict

load_dotenv() # This loads variables from .env

open_api_key = os.getenv("OPEN_AI_KEY")

def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
        model="openai/gpt-4o-mini",
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content

In [2]:
def extract_code_block(response: str) -> str:
    """Extract code block from response"""
    if not '```' in response:
        return response
    
    code_block = response.split('```')[1].strip()
    if code_block.startswith("python"):
        code_block = code_block[6:]

    return code_block

def develop_custom_function():
    # Get user input for function description
    print("\nWhat kind of function would you like to create ?")
    print("Example: 'A function that calculated the factorial of a number'")
    print("Your description: ", end="")
    function_description = input().strip()

    messages = [
        {
            "role": "system",
            "content": "You are a Python expert helping to develop a function."
        }
    ]

    messages.append({
        "role": "user",
        "content": f"Write a Python function that {function_description}. Output the function in a ```python code block```."
    })

    initial_function = generate_response(messages)

    # parse response
    initial_function = extract_code_block(initial_function)

    print("\n=== Initial function ===")
    print(initial_function)

    # Just send code block
    messages.append({
        "role": "assistant", 
        "content": "\`\`\`python\n\n"+initial_function+"\n\n\`\`\`"
    })

    # second prompt
    messages.append({
      "role": "user",
      "content": "Add comprehensive documentation to this function, including description, parameters, "
                 "return value, examples, and edge cases. Output the function in a ```python code block```."
    })

    documented_function = generate_response(messages)
    documented_function = extract_code_block(documented_function)
    print("\n=== Documented Function ===")
    print(documented_function)

    # Add documentation response to conversation
    messages.append({"role": "assistant", "content": "\`\`\`python\n\n"+documented_function+"\n\n\`\`\`"})

    # Third prompt - add test cases
    messages.append({
      "role": "user",
      "content": "Add unittest test cases for this function, including tests for basic functionality, "
                 "edge cases, error cases, and various input scenarios. Output the code in a \`\`\`python code block\`\`\`."
    })

    test_cases = generate_response(messages)
    test_cases = extract_code_block(test_cases)
    print("\n=== Test Cases ===")
    print(test_cases)

    filename = function_description.lower()
    filename = ''.join(c for c in filename if c.isalnum() or c.isspace())
    filename = filename.replace(' ', '_')[:30] + '.py'

    with open(filename, 'w') as f:
        f.write(documented_function + '\n\n' + test_cases)

    return f"Successfully written the file:{filename}"


In [5]:
develop_custom_function()


What kind of function would you like to create ?
Example: 'A function that calculated the factorial of a number'
Your description: 
=== Initial function ===

def factorial(n):
    """Calculate the factorial of a non-negative integer n."""
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers.")
    elif n == 0 or n == 1:
        return 1
    else:
        return n * factorial(n - 1)

# Example usage:
# print(factorial(5))  # Output: 120

=== Documented Function ===

def factorial(n):
    """
    Calculate the factorial of a non-negative integer n.

    The factorial of a non-negative integer n is the product of all positive integers less than 
    or equal to n. It is denoted as n! and is defined as follows:
    
        n! = n * (n - 1) * (n - 2) * ... * 3 * 2 * 1
        0! is defined to be 1.

    Parameters:
    n (int): A non-negative integer for which the factorial is to be calculated.

    Returns:
    int: The factorial of the given integer n.



'Successfully written the file:a_function_that_calculated_the.py'